In [1]:
import os

In [2]:
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\MLOPS\\MLOPS-Wine-Quality-Prediction\\research'

In [3]:
print(f"current dir")
curr_dir = %pwd
print(f"{curr_dir}")

print(f"project dir")
os.chdir("../")
%pwd

current dir
d:\college_work\4th year\Sem7th\Project\MLOPS\MLOPS-Wine-Quality-Prediction\research
project dir


'd:\\college_work\\4th year\\Sem7th\\Project\\MLOPS\\MLOPS-Wine-Quality-Prediction'

In [4]:
%%bash 
export MLFLOW_TRACKING_URI=https://dagshub.com/ukantjadia/MLOPS-Wine-Quality-Prediction.mlflow
export MLFLOW_TRACKING_USERNAME=ukantjadia 
export MLFLOW_TRACKING_PASSWORD=cb7df2a64a9e62c2f3b5a892a1e0593545a8ea77

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ukantjadia/MLOPS-Wine-Quality-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ukantjadia" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="cb7df2a64a9e62c2f3b5a892a1e0593545a8ea77"

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str 
    mlflow_uri: str

In [7]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories,save_json 

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/ukantjadia/MLOPS-Wine-Quality-Prediction.mlflow"
        )
        return model_evaluation_config

In [9]:
import pandas as pd
import os
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib

In [17]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
        
    def eval_matrix(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2 
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_matrix(test_y,predicted_qualities)
            
            # saving the matrics in json file
            scores = {"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")

In [18]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-13 09:48:56,625: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-13 09:48:56,628: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-13 09:48:56,636: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-13 09:48:56,637: INFO: common: created directory at: artifacts]
[2023-12-13 09:48:56,639: INFO: common: created directory at: artifacts/model_evaluation]
size fo all test data (1310, 13)
size of test_x and tet y (1310, 12)    (1310, 1)


c:\Users\Ukant\anaconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Ukant\anaconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/12/13 09:49:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 11
Created version '11' of model 'ElasticnetMod